# Group assignment SVD Movielens Dataset
Pip Velthuijs, 500831930
(and Jelle Schelvis)

In [1]:
import pandas as pd
import numpy as np
!pip install fastparquet
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
#loading in and showing the dataset
df = pd.read_parquet(r"C:\Users\pipve\OneDrive\Documenten\Master\System development\ml-1m\filtered_df.parquet")
df

,UserID,MovieID,Rating,Title,Action,Adventure,Animation,Children's,Comedy,Crime,...,Occupation_other or not specified,Occupation_programmer,Occupation_retired,Occupation_sales/marketing,Occupation_scientist,Occupation_self-employed,Occupation_technician/engineer,Occupation_tradesman/craftsman,Occupation_unemployed,Occupation_writer
index,,,,,,,,,,,,,,,,,,,,,
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),0,0,0,0,0,0,...,False,False,False,False,False,True,False,False,False,False
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),0,0,0,0,0,0,...,False,True,False,False,False,False,False,False,False,False
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991639,5841,3445,3,Eyes of Laura Mars (1978),0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
991640,5852,3445,2,Eyes of Laura Mars (1978),0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
991641,5916,3445,3,Eyes of Laura Mars (1978),0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,True


the following code ensures that the data is properly split for training, testing, and validation while maintaining the distribution of key features (movieid and userid) and ratings to ensure fair evaluation and prevent bias in the machine learning model. It also performs checks to ensure the consistency of data across different sets.

In [3]:
sample_size = 50000 #setting the sample size

# Performing stratified sampling based on MovieID
stratified_sampled_data = df.groupby('MovieID', group_keys=False, as_index=False).apply(lambda x: x.sample(min(len(x), sample_size))) 

# Splitting the stratified sampled data into train, test, and validation sets based on MovieID
train_temp, remaining = train_test_split(stratified_sampled_data, test_size=0.2, stratify=stratified_sampled_data['MovieID'], random_state=42)
test_data, validation_data = train_test_split(remaining, test_size=0.5, stratify=remaining['MovieID'], random_state=42)

# Further splitting based on UserID to ensure equal distribution of users
train_data, test_data = train_test_split(train_temp, test_size=len(test_data), stratify=train_temp['UserID'], random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=len(validation_data), stratify=train_data['UserID'], random_state=42)

# Ensuring similar distribution of ratings in train, test, and validation sets
rating_distribution = df['Rating'].value_counts(normalize=True)
train_distribution = train_data['Rating'].value_counts(normalize=True)
test_distribution = test_data['Rating'].value_counts(normalize=True)
validation_distribution = validation_data['Rating'].value_counts(normalize=True)

# printing out the distributions
print("Original Rating Distribution:")
print(rating_distribution)
print("\nTrain Rating Distribution:")
print(train_distribution)
print("\nTest Rating Distribution:")
print(test_distribution)
print("\nValidation Rating Distribution:")
print(validation_distribution)

# Then I check if MovieID values are aligned across train, test, and validation sets
train_movie_ids = train_data['MovieID'].unique()
test_movie_ids = test_data['MovieID'].unique()
validation_movie_ids = validation_data['MovieID'].unique()

if set(train_movie_ids) == set(test_movie_ids) == set(validation_movie_ids):
    print("MovieID values are aligned across train, test, and validation sets.")
else:
    print("Warning: MovieID values are not aligned across train, test, and validation sets.")

# I do the same for check for UserID values 
train_user_ids = train_data['UserID'].unique()
test_user_ids = test_data['UserID'].unique()
validation_user_ids = validation_data['UserID'].unique()

if set(train_user_ids) == set(test_user_ids) == set(validation_user_ids):
    print("UserID values are aligned across train, test, and validation sets.")
else:
    print("Warning: UserID values are not aligned across train, test, and validation sets.")


Original Rating Distribution:
Rating
4    0.352981
3    0.259070
5    0.232688
2    0.103836
1    0.051425
Name: proportion, dtype: float64

Train Rating Distribution:
Rating
4    0.353269
3    0.258342
5    0.232842
2    0.104116
1    0.051432
Name: proportion, dtype: float64

Test Rating Distribution:
Rating
4    0.351433
3    0.259841
5    0.233080
2    0.104557
1    0.051088
Name: proportion, dtype: float64

Validation Rating Distribution:
Rating
4    0.352768
3    0.260710
5    0.231909
2    0.103451
1    0.051162
Name: proportion, dtype: float64
MovieID values are aligned across train, test, and validation sets.
UserID values are aligned across train, test, and validation sets.


The MovieID's and UserID's are aligned. That is good. Now I can move on to converting the data into a user-item matrix which is needed for SVD.

In [4]:
# Converting into a user-item matrix and filling NaN with 0
user_item_matrix = train_data.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)
matrix_validation = validation_data.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)
matrix_test = test_data.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)
user_item_matrix

MovieID,1,2,3,4,5,6,7,9,10,11,...,3927,3928,3929,3930,3932,3936,3937,3948,3949,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Then I convert the matrices to numpy. SVD involves matrix computations, and numpy arrays provide optimized routines for these computations, making the process faster and more memory-efficient. 

In [5]:
# Converting the user-item matrix to a numpy array
matrix_train_array = np.array(user_item_matrix)
matrix_validation_array = np.array(matrix_validation)
matrix_test_array = np.array(matrix_test)

After this, I want to make sure the shape of the matrices are correct. Therfore, I print the shapes per mattrix.

In [6]:
# To get the shape of each matrix
shape_train = matrix_train_array.shape
shape_validation = matrix_validation_array.shape
shape_test = matrix_test_array.shape

print(f"Shape of matrix_train_array: {shape_train}")
print(f"Shape of matrix_validation_array: {shape_validation}")
print(f"Shape of matrix_test_array: {shape_test}")

Shape of matrix_train_array: (6040, 2006)
Shape of matrix_validation_array: (6040, 2006)
Shape of matrix_test_array: (6040, 2006)


The shapes are aligned, so that's good.

In [7]:
def manual_svd(matrix_train_array):
    # Computing matrix_train * matrix_train^T and matrix_train^T * matrix_train
    AAT = np.dot(matrix_train_array, matrix_train_array.T)
    ATA = np.dot(matrix_train_array.T, matrix_train_array)
    
    # Finding eigenvalues and eigenvectors for AAT and ATA
    eigenvalues_U, U = np.linalg.eigh(AAT)
    eigenvalues_V, V = np.linalg.eigh(ATA)
    
    #Sorting the eigenvectors of U and V by decreasing eigenvalues
    idx_U = eigenvalues_U.argsort()[::-1]
    U = U[:,idx_U]
    
    idx_V = eigenvalues_V.argsort()[::-1]
    V = V[:,idx_V]
    
    # compose singular values, that are the square roots of the eigenvalues of AAT or ATA
    singular_values = np.sqrt(np.abs(eigenvalues_U[idx_U]))
    
    # Constructing the Sigma matrix that is formed using the singular values calculated above
    min_dim = min(U.shape[0], V.shape[0])
    Sigma = np.zeros((U.shape[0], V.shape[0]))
    np.fill_diagonal(Sigma, singular_values[:min_dim])
    
    # Then i compute V^T
    VT = V.T
    
    return U, Sigma, VT


the function returns U sigma and VT, which together represent the components of the SVD decomposition of the input matrix matrix_train_array

In [8]:
# Decomposing matrix_train
U, Sigma, VT = manual_svd(matrix_train_array)

After this, I have three matrices:

U: This matrix represents the left singular vectors of the SVD decomposition of matrix_train_array. Each column of U corresponds to a left singular vector.

Sigma: This is a diagonal matrix containing the singular values of the SVD decomposition. The singular values are arranged in decreasing order along the diagonal.

VT: This matrix represents the right singular vectors of the SVD decomposition of matrix_train_array. Each row of VT corresponds to a right singular vector.

The following lines of code are making functions which will later on be used:

In [9]:
# here I use a function to reconstruct the matrix from U, sigma and VT with the top k singular values
def reconstruct_matrix(U, Sigma, VT, k):
    U_k = U[:, :k]
    Sigma_k = Sigma[:k, :k]
    VT_k = VT[:k, :]
    return np.dot(U_k, np.dot(Sigma_k, VT_k))

In [10]:
# here I calculate the RMSE between the actual and predicted matrices
def calculate_rmse(actual, predicted):
    difference = actual - predicted 
    # Only considering non-zero entries (so it is ignoring missing values in the validation set)
    nonzero_indices = np.nonzero(actual)
    difference_filtered = difference[nonzero_indices] # Filtering difference array to consider only non-zero entries
    rmse = np.sqrt(np.mean(np.square(difference_filtered))) # Calculating RMSE by taking the square root of the mean of squared differences
    return rmse #returning rmse

After defining the funcitons, I perform hyperparameter search to find the best value of k (the number of singular values to retain in the SVD) by reconstructing the matrix for each value of k, calculating the RMSE between the reconstructed matrix and the validation matrix, and then selecting the k value that has the lowest rmse.

In [25]:
# Now, perform the grid search over k values and calculate RMSE
rmse_values = []
k_values = list(range(5, 50, 5))

for k in k_values:
    # Reconstructing the matrix with current k
    matrix_pred = reconstruct_matrix(U, Sigma, VT, k)
    # Calculating RMSE for the current k
    rmse = calculate_rmse(matrix_validation_array, matrix_pred)
    rmse_values.append(rmse)
    print(f"RMSE for k={k}: {rmse}")

# Finding the best k
min_rmse = min(rmse_values)
best_k = k_values[rmse_values.index(min_rmse)]
print(f"Best k by RMSE: {best_k} with RMSE: {min_rmse}")

RMSE for k=5: 4.386734104908001
RMSE for k=10: 4.412128280427256
RMSE for k=15: 4.4056957957917
RMSE for k=20: 4.405285908139326
RMSE for k=25: 4.406917984400658
RMSE for k=30: 4.410173605998855
RMSE for k=35: 4.404811348552096
RMSE for k=40: 4.406931487559092
RMSE for k=45: 4.414751362441108
Best k by RMSE: 5 with RMSE: 4.386734104908001


The outcome shows the best k is close to 5. Let's find the exact number of k.

In [26]:
# New range for k between 2 and 20
k_values = range(2, 20)  

# Here I initialize an empty list to store rmse values
rmse_values = []

for k in k_values:
    # Reconstructing the matrix with current k
    matrix_pred = reconstruct_matrix(U, Sigma, VT, k)
    # Calculating RMSE for the current k using the adjusted validation matrix
    rmse = calculate_rmse(matrix_validation_array, matrix_pred)
    rmse_values.append(rmse)
    print(f"RMSE for k={k}: {rmse}")

# Finding the best k
min_rmse = min(rmse_values)
best_k = k_values[rmse_values.index(min_rmse)]
print(f"Best k by RMSE: {best_k} with RMSE: {min_rmse}")

RMSE for k=2: 4.489056258085771
RMSE for k=3: 4.442758708108489
RMSE for k=4: 4.410088890143995
RMSE for k=5: 4.386734104908001
RMSE for k=6: 4.414654526864536
RMSE for k=7: 4.3954906402701654
RMSE for k=8: 4.411719423776666
RMSE for k=9: 4.39900629074798
RMSE for k=10: 4.412128280427256
RMSE for k=11: 4.405626691757673
RMSE for k=12: 4.399851435539164
RMSE for k=13: 4.404653332988082
RMSE for k=14: 4.40279440497532
RMSE for k=15: 4.4056957957917
RMSE for k=16: 4.401869332568574
RMSE for k=17: 4.404181679347844
RMSE for k=18: 4.404154607958718
RMSE for k=19: 4.404313559772763
Best k by RMSE: 5 with RMSE: 4.386734104908001


the rmse is kind of high. This is probably because there are not used anymore features. 

In [27]:
# Reconstructing the matrix with the best k found using the validation set
best_matrix_pred = reconstruct_matrix(U, Sigma, VT, best_k)

# Calculating RMSE on the test set using the best k
test_rmse = calculate_rmse(matrix_test_array, best_matrix_pred)
print(f"RMSE on Test Set for best k={best_k}: {test_rmse}")

RMSE on Test Set for best k=5: 4.382909154772543


Now, i can use the reconstructed matrix to make recommendations for users. For example, recommending movies for a specific user by sorting the predicted ratings for that user and recommending the movies with the highest predicted ratings that the user hasn't seen.

In [38]:
def recommend_movies_for_user(user_id, reconstructed_matrix, df, top_n=5):
    """
    Recommends top n movies for a given user based on the reconstructed matrix.
    """
    # Get the row corresponding to the user_id
    user_row = reconstructed_matrix[user_id - 1]
    # Get indices of movies the user hasn't rated (rated as 0)
    unrated_movies_indices = np.where(user_row == 0)[0]
    # Get MovieIDs for unrated movies
    unrated_movie_ids = df.columns[unrated_movies_indices]
    # Sort the unrated movies based on predicted ratings
    recommended_movies = sorted(zip(unrated_movie_ids, user_row[unrated_movies_indices]), key=lambda x: x[1], reverse=True)
    # Return top n recommended movies
    return recommended_movies[:top_n]

# Example: Recommend movies for user 1
user_id = 23
top_recommendations = recommend_movies_for_user(user_id, best_matrix_pred, user_item_matrix)
print(f"Top 5 movie recommendations for User {user_id}:")
for i, (movie_id, rating) in enumerate(top_recommendations, 1):
    print(f"{i}. MovieID: {movie_id}, Predicted Rating: {rating}")

Top 5 movie recommendations for User 23:


---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------